In [5]:
import Pkg;
Pkg.activate(@__DIR__)
Pkg.status()

Status `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/distributed/Project.toml`
  [d58978e5] Dagger v0.18.12
  [aaf54ef3] DistributedArrays v0.6.7


  Activating project at `/global/u1/b/blaschke/juliacon24-hpcworkshop/parts/distributed`


In [6]:
using Distributed

addprocs(4)

4-element Vector{Int64}:
 2
 3
 4
 5

In [10]:
@everywhere using Dagger

In [11]:
DX = rand(Blocks(50, 50), 100, 100)

100x100 DMatrix{Float64} with 2x2 partitions of size 50x50:
~0% completed
 ...  ...  ...  ...  ...  …  ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...  …  ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...  …  ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ⋮                                  ⋱  ⋮                           
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...     ...  ...  ...  ...  ...
 ...  ...  ...  ...  ...  …  ...  ...  ...  ...  ...
 ...  ... 

In [12]:
DX

100x100 DMatrix{Float64} with 2x2 partitions of size 50x50:
 0.287058   0.932143   0.893901   …  0.787941   0.281469    0.797844
 0.0780438  0.117807   0.623861      0.726202   0.518999    0.108689
 0.209235   0.664006   0.843917      0.995339   0.128314    0.924156
 0.667049   0.962158   0.952806      0.505621   0.902641    0.35117
 0.422007   0.958753   0.312819      0.0161274  0.578171    0.271552
 0.977362   0.635106   0.199905   …  0.909914   0.103151    0.552586
 0.176873   0.121315   0.967516      0.633998   0.471396    0.80213
 0.773225   0.139292   0.0816662     0.176016   0.33704     0.365034
 0.708157   0.367415   0.830456      0.220995   0.329504    0.0199784
 0.85119    0.249636   0.565433      0.2289     0.972173    0.631671
 0.96518    0.679127   0.399155   …  0.391515   0.00204959  0.403234
 0.435758   0.629236   0.336337      0.976523   0.371747    0.200271
 0.670513   0.873129   0.715954      0.964522   0.225559    0.205521
 ⋮                                ⋱         

In [17]:
DX.subdomains

2×2 Dagger.DomainBlocks{2}:
 ArrayDomain{2, Tuple{UnitRange{Int64}, UnitRange{Int64}}}((1:50, 1:50))    …  ArrayDomain{2, Tuple{UnitRange{Int64}, UnitRange{Int64}}}((1:50, 51:100))
 ArrayDomain{2, Tuple{UnitRange{Int64}, UnitRange{Int64}}}((51:100, 1:50))     ArrayDomain{2, Tuple{UnitRange{Int64}, UnitRange{Int64}}}((51:100, 51:100))